In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Building a neural network from scratch
This is a personal project, my intent is to get a better understanding of NN and how they "learn".

## What is a neural network?

Neural networks are a type of algorithm inspired by the way brains work. They are constructed by three main parts, the __layers__: 
* The input layer: The data we'll feed to the NN
* The hidden layers: A collection of functions that process the data, where the network finds paterns
* The output layer: The value or class we are trying to predict

Here's a definition I took from IBM's website: 

_Neural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are a subset of machine learning and are at the heart of deep learning algorithms. Their name and structure are inspired by the human brain, mimicking the way that biological neurons signal to one another._

_Artificial neural networks (ANNs) are comprised of node layers, containing an input layer, one or more hidden layers, and an output layer. Each node, or artificial neuron, connects to another and has an associated weight and threshold. If the output of any individual node is above the specified threshold value, that node is activated, sending data to the next layer of the network. Otherwise, no data is passed along to the next layer of the network._

## Nodes and Weights
__Nodes__ are what constitude each layer. Each node in any given layer is linked with all of the next layer's nodes. Think about it this way: every two layers in a NN is a fully conected directed bipartite network. These links come in the form of what we call __weights__, which are a number that is multiplied by the link's parent node value. So the receiving node has three values, each one is the value of the previous node multiplied by the weight, and they are all summed up, plus a bias, to form the value of this node.  

In [ ]:
# Example
# Here, 7, 3 and 2 are individual nodes
input_layer = [7, 3, 2]
# These are the values each node will be multiplied by
weights = [0.1, -0.6, 0.5]
# The bias will be added at the end
bias = 1

new_node = (input_layer[0]*weights[0] + input_layer[1]*weights[1] + input_layer[2]*weights[2]) + bias
print(round(new_node, 1))

## Computing the value of nodes
The purose of the following code is to make the calculations of the next layer easy to see. Further down we'll start using Numpy to run the code faster with less typing. 

In [ ]:
# Let's do the same, but this time with four nodes in the input layer (HL) and three nodes in the output layer. 
# We will simulate the output layer, coming from the last hidden layer. 

HL = [7, 3, 2, 3]

# Weights for the first node
W1 = [0.1, -0.6, 0.5, 0.8]
# For the second
W2 = [0.7, 0.4, 0.1, -0.6]
# For the third
W3 = [0.5, -0.6, -0.8, 0.13]

bias1 = 5
bias2 = 7
bias3 = 5

output = [HL[0]*W1[0] + HL[1]*W1[1] + HL[2]*W1[2] + HL[3]*W1[3] + bias1,
         HL[0]*W2[0] + HL[1]*W2[1] + HL[2]*W2[2] + HL[3]*W2[3] + bias2,
         HL[0]*W3[0] + HL[1]*W3[1] + HL[2]*W3[2] + HL[3]*W3[3] + bias3]

print(list(map(lambda x: round(x,2), output)))

Tweaking the weights and the biases is how the NN is able to generalize and make decisions. 

In [ ]:
# Lets clean the previous code

HL = np.array([7, 3, 2, 3])

weights = np.array([[0.1, -0.6, 0.5, 0.8],
          [0.7, 0.4, 0.1, -0.6],
          [0.5, -0.6, -0.8, 0.13]])

biases = np.array([5, 7, 5])

output = []
# Loop through the weights and biases corresponding to each output node
# This first loop runs for 3 iterations, one for every output node
for weight, bias in zip(weights, biases):
    node_output = 0
    # Loop through the inputs and the weight of this loop's node
    # Compute the product of each input by its weight and add the result to the node
    for n_input, weight1 in zip(HL, weight):
        node_output += n_input*weight1
    # Add the bias to the node and append it to the list
    node_output += bias
    output.append(node_output)
    # Repeat for the rest of the nodes
    
list(map(lambda x: round(x,2), output))

### Batches
We can speed up the computing time by taking advantage of computers' capacity to compute in parallel

In [ ]:
inputs = np.array([[7, 3, 2, 3],
                   [3, 6, 9, 1],
                   [3, 7, 1, 9]])

weights = np.array([[0.1, -0.6, 0.5, 0.8],
                    [0.7, 0.4, 0.1, -0.6],
                    [0.5, -0.6, -0.8, 0.13]])

biases = [5, 7, 5]
# The commented out line of code won't work because the rows of inputs doesnt align with the columns of weights
#np.dot(inputs, weights)
# We need to compute each row of input to the weights, so we have to transpose the input matrix for this to work

np.dot(inputs, weights.T)+biases
# Now each row corresponds to an output

This new matrix can then be fed to the next layer, we can do the same operations using new weights and biases

In [ ]:
np.random.seed(1)
inputs1 = np.array([[7, 3, 2, 3],
                   [3, 6, 9, 1],
                   [3, 7, 1, 9]])

weights1 = np.array([[0.1, -0.6, 0.5, 0.8],
                    [0.7, 0.4, 0.1, -0.6],
                    [0.5, -0.6, -0.8, 0.13]])

biases1 = [5, 7, 5]

inputs2 = np.random.randn(3,4)

weights2 = np.random.randn(3,4)

biases2 = np.random.randint(0, 9, 3)

first_layer = np.dot(inputs1, weights1.T)+biases1
print(first_layer.shape)
second_layer = np.dot(inputs2, weights2.T)+biases2
second_layer

## Objects
It'll be a lot easier to work with the network if we create objects. This part of the notebook I took from Sentex's YouTube tutorial on Neural Networks from scratch

In [ ]:
np.random.seed(0)
X = np.array([[1, 2, 3, 2.5],
              [2, 5, -1, 2],
              [-1.5, 2.7, 3.3, -0.8]])

class Layer_Dense:
    # We'll need to define what are the number of inputs 
    # and what are the number of neurons they are conected to
    def __init__(self, n_inputs, n_neurons):
        # multiply by 0.1 to keep values small
        self.weights = 0.1*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

# The first parameter is the number of inputs to this layer
# the second is the number of neurons we want to create (any number we want)
layer1 = Layer_Dense(X.shape[1], 5)
layer1.forward(X)
print(layer1.output, '\n')

# We can then use the output of the first step as the input of the second
# We just need to provide the number of inputs (through the shape of the last output),
# and the number of neurons we want (any number, 2 in this case)
layer2 = Layer_Dense(layer1.output.shape[1], 2)
layer2.forward(layer1.output)
print(layer2.output)

## Activation functions
They are functions that reside in every node of the hidden layers. Their job is to output certain values, given a certain input. In this first example, we'll use a _step function_, which just outputs 0s and Xs (original values), depending on wether the input is > 0 or < 0. Output layers usually have a different activation function. Other types of activation functions: sigmoid function, rectified linear function...

In [ ]:
class Activation_ReLU:
    # This is all there is to the activation function. 
    # If it's zero or less, output 0. If it's more than 0, output the input.
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [ ]:
layer1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()
layer1.forward(X)
print(layer1.output, '\n')
activation1.forward(layer1.output)
print(activation1.output)

## SoftMax Activation function  
We have to determine how wrong each prediction is. The way we have done things so far makes the outputs susceptible to be greater than 1. We could, for example, have [1.5, 4.6, 0.89] as the outputs. And of course, we could infer from this that the algorithm has decided that the second output is the more probable, but in order to make backpropagation work, we need to have standardized outputs, because we need to calculate a loss function at the end of every iteration. 

In [ ]:
# Example

import math

layer_outputs = [4.8, 1.21, 2.385]

# Get rid of negative values
e = math.e
exp_outputs = [e**i for i in layer_outputs]

# Scale to "probabilities"
summed = sum(exp_outputs)
output = [i/summed for i in exp_outputs]
print(f'With raw Python: {output}')

# With numpy

exp_outputs = np.exp(layer_outputs)
output = exp_outputs/np.sum(exp_outputs)
# These two lines of code is what we call the SoftMax activation function

print(f'With numpy: {output}')

In [ ]:
layer_outputs = [[4.8, 1.21, 2.385],
                [8.9, -1.81, 0.2],
                [1.41, 1.051, 0.026]]

exp_outputs = np.exp(layer_outputs)

# To be able to sum the rows of the exp_outputs matrix, we can tweak the parameters of np.sum()
print(np.sum(exp_outputs, axis=1))
x = exp_outputs/np.sum(exp_outputs, axis=1)
print(f'Sums: {np.sum(x, axis=1)}', '\n')

# But that gives out a vector which sum doesnt equal 1. We would like to keep the sums in their rows so 
# we can divide the exp_outputs matrix by the sums automaticaly like before
print(np.sum(exp_outputs, axis=1, keepdims=True))
x = exp_outputs/np.sum(exp_outputs, axis=1, keepdims=True)
print(f'Sums: {np.sum(x, axis=1)}')

In [ ]:
# To avoid overflow erros, where the number being computed is too large
# we can substract all the numbers of the input by the largest input.
# This will cause the exp function to get fixed between 0 and 1,
# and the result doesnt change because we apply the same operation to every input.

# We'll do this as we create the SoftMax object

class SoftMax:
    def forward(self, inputs):
        # np.max(inputs) would output a single number
        # Tweaked the parameters just as before
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values/np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [ ]:
# This is a module Sentex created to generate data

import nnfs
from nnfs.datasets import spiral_data
nnfs.init()

X, y = spiral_data(samples=100, classes=3)

In [ ]:
class Layer_Dense:
    # We'll need to define what are the number of inputs 
    # and what are the number of neurons they are conected to
    def __init__(self, n_inputs, n_neurons):
        # multiply by 0.1 to keep values small
        self.weights = 0.1*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        
class Activation_ReLU:
    # This is all there is to the activation function. 
    # If it's zero or less, output 0. If it's more than 0, output the input.
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
    
class SoftMax:
    def forward(self, inputs):
        # np.max(inputs) would output a single number
        # Tweaked the parameters just as before
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values/np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [ ]:
dense1 = Layer_Dense(2, 3)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(3, 3)
activation2 = SoftMax()

dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)
dense1.output[:5]

## The loss function: Categorical cross-entropy
This function takes the natural logarithm of the predicted probabilities of each class and multiplies it by the corresponding One-Hot encoded vector value (see the example below for clarification) and adds them up.

Since the only non-zero value of the encoding is the actual class, the formula for cross-entropy simplyfies a lot: it is the negative log of the highest predicted probability for each iteration. 

In [ ]:
# Let's say the predicted class is the first one

predictions = [0.7, 0.25, 0.05]

OneHot_Encoding = [1, 0, 0]

CrossEntropyLoss = [-math.log(predictions[0])*OneHot_Encoding[0],
                   -math.log(predictions[1])*OneHot_Encoding[1],
                   -math.log(predictions[2])*OneHot_Encoding[2]]
print(sum(CrossEntropyLoss))
# Notice this is the same thing as 
print(-math.log(predictions[0]), '\n')
# Because the rest just cancels out
# Also, notice how the loss increases as the predicted value is more "wrong"
for i in predictions:
    print(-math.log(i))

The loss increases exponentially as the predicted value approaches 0. This will help us penalize weights and biases that are performing poorly even more than mearly taking 1-p. 

In [ ]:
x = np.linspace(0,1, 1000)
x[0] = 0.00001
y = [-math.log(i) for i in x]
plt.figure(figsize=(9,6))
plt.plot(x, y, label='Cross-Entropy')
plt.plot(x, [1-i for i in x], label='1-p')
plt.xlabel('Predictions')
plt.ylabel('Loss')
plt.title('Cross-Entropy compared to 1-p as a loss function')
plt.annotate('Cross-Entropy excels at penalizing the worst predictions', xy=(0.2,7), size='large')
plt.legend();

## Implementing the loss function
We are passing a batch of samples to the NN, so we will also have a batch of predictions at the end of each iteration, and that batch we'll need to evaluate against a batch of _actual_ classes.

In [ ]:
# Let's say this is our output, one line for each sample

output = np.array([[0.7, 0.1, 0.2],
                   [0.1, 0.5, 0.4],
                   [0.02, 0.9, 0.08]])

# And let's say this is our y, our actual class vector
y = np.array([0, 1, 1])
# Where each value corresponds to the actual class of the row of our output
# In our output:
# the first row's actual value is 0: so the value we want is 0.7
# the second row's actual value is 1: so we want 0.5
# and the third row's actual value is 1: so we want 0.9

So in our example, the desired result would be [0.7, 0.5, 0.9]; the confidence of the model for the actual classes.
We could write a loop to use the values in "y" as the index we need from each line of our output matrix, but it'd be faster and more compact to do this with numpy.

All we have to do is use lists as the indeces for rows and columns. We can iterate through all the rows and pass the "y" vector as the columns:

In [ ]:
print(output[[0, 1, 2], [0, 1, 1]])

# Same as

print(output[[0, 1, 2], y])

# Same as

print(output[range(len(output)), y])

Now we just need to pass these values through our loss function

In [ ]:
-np.log(output[
    range(len(output)),
    y
])

## Backpropagation

This is the messier part of my notebook so far. Backpropagation is very dense, both theorically and practically. I spent a week watching tutorials, reading, and writting down the derivates of each of the functions of various NN architectures. 

This is where I had to stop, because my semester is starting soon and I need to focus. But I want to finish this project, seeing the loss function converge to near zero was one of the most satisfing things I have ever experienced. 

My implementation is certainly not optimal, and my loss function has weird spikes as it goes down to the optimum value. But I did it on my own, and got a very good look into the inner workings of Neural Networks.

In [ ]:
class Layer_Dense:
    # We'll need to define what are the number of inputs 
    # and what are the number of neurons they are conected to
    def __init__(self, n_inputs, n_neurons):
        # multiply by 0.1 to keep values small
        self.weights = 0.1*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        
class Activation_ReLU:
    # This is all there is to the activation function. 
    # If it's zero or less, output 0. If it's more than 0, output the input.
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
        
class SoftMax:
    def forward(self, inputs):
        # np.max(inputs) would output a single number
        # Tweaked the parameters just as before
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values/np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1/(1+np.exp(-inputs))
        

class Loss:
    def calculate(self, output, y):
        self.sample_loss = self.forward(output, y)
        mean_loss = np.mean(self.sample_loss)
        return mean_loss
    
# CCE = Categorical cross-entropy
class Loss_CCE(Loss):
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        # We can have actual values in the form of one-hot encoded list of lists
        # or we could have just one list with the value for each class
        # We need to take both formats in consideration
        
        # If y_true is just a vector
        # This is the case we worked on, in the previous example
        if len(y_true.shape) == 1:
            self.correct_confidence = y_pred_clipped[range(len(y_pred_clipped)), y_true]
        # This is a matrix
        # One-Hot encoded
        elif len(y_true.shape) == 2:
            # Element-wise multiplication of the 2 matrices
            # y_true has one row of 0s and a 1 for every sample
            # So the multiplication only occurs on the predicted value of the actual class
            # Then np.sum() "sums" (there is only one non-zero value) the values along the rows
            self.correct_confidence = np.sum(y_pred_clipped*y_true, axis=1)
        else: correct_confidence = y_pred[0][y_true]
        
        negative_log_likelihoods = -np.log(self.correct_confidence)
        return negative_log_likelihoods            

In [ ]:
# Create data and pass it through the untrained NN
np.random.seed(0)
X, y = spiral_data(samples=100, classes=2)

dense1 = Layer_Dense(2, 2)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(2, 2)
activation2 = SoftMax()

loss_function = Loss_CCE()


dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

loss = loss_function.calculate(activation2.output, y)
print(loss)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y);

In [ ]:
np.random.seed(0)

samples = 100
classes = 3
m = samples*classes

X, y = spiral_data(samples=samples, classes=classes)

dense1 = Layer_Dense(2, 10)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(10, classes)
activation2 = SoftMax()

loss_function = Loss_CCE()
####################
losses = []
alpha = 0.1
for i in range(3000):

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    loss = loss_function.calculate(activation2.output, y)

    OH_y = np.zeros(activation2.output.shape)
    OH_y[range(len(activation2.output)), y] = 1
    
    dSM = activation2.output - OH_y
    dw2 = np.dot(activation1.output.T, dSM)/m
    da1 = np.dot(dense2.weights, dSM.T)*np.greater(activation1.output, 0).astype(int).T
    dw1 = np.dot(X.T, da1.T)/m
    
    b2 = np.subtract(dense2.biases, (alpha*dSM.mean(axis=0)))
    w2 = np.subtract(dense2.weights, (alpha*dw2))
    b1 = np.subtract(dense1.biases, (alpha*da1.mean(axis=1, keepdims=True).T))
    w1 = np.subtract(dense1.weights, (alpha*dw1))

    dense2.biases = b2
    dense2.weights = w2
    dense1.biases = b1
    dense1.weights = w1

    losses.append(loss)
    if i%10000 == 0:
        print(f'Loss on the {i}th iteration: {loss}')
print(loss)
plt.plot(range(len(losses)), losses);

In [ ]:
np.random.seed(0)


s = 200
c = 2
m = s*c

X, y = spiral_data(samples=s, classes=c)

dense1 = Layer_Dense(2, 100)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(100, 100)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(100, c)
activation3 = SoftMax()

loss_function = Loss_CCE()
####################
losses = []
alpha = 0.5
for i in range(3000):

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss = loss_function.calculate(activation3.output, y)

    OH_y = np.zeros(activation3.output.shape)
    OH_y[range(len(OH_y)), y] = 1
    
    dSM = activation3.output - OH_y
    da2 = np.dot(dense3.weights, dSM.T)*np.greater(activation2.output, 0).astype(int).T
    da1 = np.dot(dense2.weights, da2)*np.greater(activation1.output, 0).astype(int).T
    dw3 = np.dot(activation2.output.T, (dSM))/m
    dw2 = np.dot(activation1.output.T, da2.T)/m
    dw1 = np.dot(da1, X).T/m
    
    b3 = np.subtract(dense3.biases, (alpha*dSM.mean(axis=0)))
    w3 = np.subtract(dense3.weights, (alpha*dw3))
    b2 = np.subtract(dense2.biases, (alpha*da2.mean(axis=1)))
    w2 = np.subtract(dense2.weights, (alpha*dw2))
    b1 = np.subtract(dense1.biases, (alpha*da1.mean(axis=1)))
    w1 = np.subtract(dense1.weights, (alpha*dw1))
    
    dense3.biases = b3
    dense3.weights = w3
    dense2.biases = b2
    dense2.weights = w2
    dense1.biases = b1
    dense1.weights = w1

    losses.append(loss)
    
    if i%1000 == 0:
        print(f'Loss on the {i}th iteration: {loss}')
        
print(loss)
plt.plot(losses);

I still have to figure out why I get those spikes during training, but it is still satisfying to see the NN converge to an optimum.

In [ ]:
# We can graph the decision boundary of the trained NN

min1, max1 = X[:, 0].min()-.1, X[:, 1].max()+.1
min2, max2 = X[:, 0].min()-.1, X[:, 1].max()+.1

x1_scale = np.arange(min1, max1, 0.01)
x2_scale = np.arange(min2, max2, 0.01)

x_grid, y_grid = np.meshgrid(x1_scale, x2_scale)

x_g, y_g = x_grid.flatten(), y_grid.flatten()
x_g, y_g = x_g.reshape(len(x_g),1), y_g.reshape(len(y_g),1)

grid = np.hstack((x_g, y_g))

grid1 = np.array([np.arange(-1,1,0.0001), np.arange(-1,1,0.0001)]).T

dense1.forward(grid)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

op = activation3.output

pred = np.greater(op, 0.5)

plt.scatter(X[:,0], X[:,1], c=y)
plt.contourf(x_grid, y_grid, pred[:, 1].reshape(x_grid.shape), alpha=0.25);